<a href="https://colab.research.google.com/github/tourihasi/Openstudio/blob/main/%E9%83%A8%E5%B1%8B%E3%81%A8%E7%A9%BA%E8%AA%BF%E7%B3%BB%E7%B5%B1%E3%81%AE%E7%A2%BA%E8%AA%8D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ① インストール & インポート（openstudio が未導入なら pip）
try:
    import openstudio as osd
except ModuleNotFoundError:
    # ①-1 Colab では '!' が必要（導入に少し時間がかかります）
    !pip -q install openstudio
    import openstudio as osd

# ② 標準ライブラリ & Colab入出力 & 表形式
import io, os
import pandas as pd
from google.colab import files

# ③ OSMファイルのアップロード
print("③ OSMファイル（.osm）を1つ選択してください...")
uploaded = files.upload()
osm_path = next((k for k in uploaded.keys() if k.lower().endswith(".osm")), None)
assert osm_path, "OSMファイル（.osm）が見つかりませんでした。"

# ④ モデルの読み込み（失敗時は例外）
opt_model = osd.model.Model.load(osd.path(osm_path))
assert opt_model.is_initialized(), f"OpenStudio Model の読み込みに失敗: {osm_path}"
m = opt_model.get()

# ⑤ ThermalZone → AirLoopHVAC の対応関係を収集
rows = []
zones = m.getThermalZones()

for z in zones:
    # ⑤-1 ゾーン名（nameStringは常に文字列を返す簡便メソッド）
    zone_name = z.nameString()

    # ⑤-2 ゾーンに紐づくAirLoopHVAC（ない場合は None）
    #     ※ 通常は1つですが、Noneの可能性もあるため存在チェック
    opt_loop = z.airLoopHVAC()
    if opt_loop.is_initialized():
        loop_name = opt_loop.get().nameString()
    else:
        loop_name = None  # 端末未接続やDOAS未割当など

    # ⑤-3 追加情報（オプション）：設計サプライ温度スケジュールなどが欲しければ loop から辿れます
    rows.append({
        "ThermalZone": zone_name,
        "AirLoopHVAC": loop_name
    })

# ⑥ DataFrame 化 & 表示
df = pd.DataFrame(rows).sort_values(by=["AirLoopHVAC", "ThermalZone"], na_position="last").reset_index(drop=True)
df

# ⑦ CSV 出力 & 自動ダウンロード
csv_name = os.path.splitext(os.path.basename(osm_path))[0] + "_Zone_AirLoop_Map.csv"
df.to_csv(csv_name, index=False, encoding="utf-8-sig")
files.download(csv_name)

# ⑧ 付記（ログ出力）：サマリー
print(f"⑧ サマリー：ゾーン数 = {len(zones)}, AirLoop未接続ゾーン数 = {df['AirLoopHVAC'].isna().sum()}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 MB 5.5 MB/s eta 0:00:00
③ OSMファイル（.osm）を1つ選択してください...


Saving 251020_1.osm to 251020_1.osm


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

⑧ サマリー：ゾーン数 = 108, AirLoop未接続ゾーン数 = 56
